In [14]:
import os

import pandas as pd
from dotenv import load_dotenv
from web3 import Web3

from notebooks import fetch_all_pools_info
from notebooks import get_abi

load_dotenv()
web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
ARBITRUM_CHAIN_LITERAL = "ARBITRUM"
ARBITRUM_TOKEN_ADDR = Web3.to_checksum_address("0x912ce59144191c1204e64559fe8253a0e49e6548")
# Fetch all pools from Balancer API
all_pools = fetch_all_pools_info("arbitrum")

# Collect arb gauges
arb_gauges = {}
for pool in all_pools:
    # Only collect gauges for the arb chain and that are not killed
    if pool['chain'] == ARBITRUM_CHAIN_LITERAL and pool['gauge']['isKilled'] is False:
        _gauge_addr = Web3.to_checksum_address(pool['gauge']['address'])
        arb_gauges[_gauge_addr] = {
            'gaugeAddress': pool['gauge']['address'],
            'pool': pool['address'],
            'symbol': pool['symbol'],
            'id': pool['id'],
            'childGauge': Web3.to_checksum_address(pool['gauge']['childGaugeAddress']),
        }
print(f"Found {len(arb_gauges)} gauges on Arbitrum")

# Now go through all gauges and check that each of them has reward tokens set to ARB
gauge_reward_data = {}
for _, pool_info in arb_gauges.items():
    child_gauge_contract = web3.eth.contract(address=pool_info['childGauge'],
                                             abi=get_abi("ChildGauge"))
    data = child_gauge_contract.functions.reward_data(ARBITRUM_TOKEN_ADDR).call()
    gauge_reward_data[pool_info['childGauge']] = {
        'pool_id': pool_info['id'],
        'symbol': pool_info['symbol'],
        'root_gauge': pool_info['gaugeAddress'],
        'child_gauge': pool_info['childGauge'],
        'distributor': data[0],
        'period_finish': data[1],
        'rate': data[2],
        'last_update': data[3],
        'integral': data[4],
    }

# Into a dataframe and print
df = pd.DataFrame.from_dict(gauge_reward_data, orient='index')
df.to_csv('arb_gauge_reward_data.csv', index=False)
df

Found 27 gauges on Arbitrum


,pool_id,symbol,root_gauge,child_gauge,distributor,period_finish,rate,last_update,integral
0x2eB5661002b68EBE887d29d415c3A3b52536912C,0x4a2f6ae7f3e5d715689530873ec35593dc28951b0000...,wstETH/rETH/cbETH,0x56c0626e6e3931af90ebb679a321225180d4b32b,0x2eB5661002b68EBE887d29d415c3A3b52536912C,0xF23d8342881eDECcED51EA694AC21C2B68440929,1699030269,7596985895294719,1699015019,18988074066872783990
0xcf9f895296F5e1D66a7D4dcf1d92e1B435E9f999,0x32df62dc3aed2cd6224193052ce665dc181658410002...,RDNT-WETH,0x8135d6abfd42707a87a7b94c5cfa3529f9b432ad,0xcf9f895296F5e1D66a7D4dcf1d92e1B435E9f999,0xF23d8342881eDECcED51EA694AC21C2B68440929,1699030269,5642679806877253,1699002558,21186921258308628
0x04fc019017eD3F921D5ec11fFf84B870744BA0d1,0xa231aea07bb5e79ae162f95903806fc5ad65ff110002...,50DFX-50WETH,0x8f7a0f9cf545db78bf5120d3dbea7de9c6220c10,0x04fc019017eD3F921D5ec11fFf84B870744BA0d1,0xF23d8342881eDECcED51EA694AC21C2B68440929,1699030269,999707198468991,1698936226,1601038301417445300
0x159be31493C26F8F924b3A2a7F428C2f41247e83,0x49b2de7d214070893c038299a57bac5acb8b8a340001...,GOLD-BAL-AURA-wstETH,0x86cf58bd7a64f2304227d1a490660d2954db4a91,0x159be31493C26F8F924b3A2a7F428C2f41247e83,0x0000000000000000000000000000000000000000,0,0,0,0
0xeF767E740D83d410794519c2F93Db32e44359a5C,0xb3028ca124b80cfe6e9ca57b70ef2f0ccc41ebd40002...,50MAGIC-50USDC,0x0edf6cdd81bc3471c053341b7d8dfd1cb367ad93,0xeF767E740D83d410794519c2F93Db32e44359a5C,0xF23d8342881eDECcED51EA694AC21C2B68440929,1699030269,4124760536570968,1699008608,19803040506945721
0xb438c6cc53315FfA3fcD1bc8b27d6c3155b0B56A,0x542f16da0efb162d20bf4358efa095b70a100f9e0000...,2BTC,0xc4b6cc9a444337b1cb8cbbdd9de4d983f609c391,0xb438c6cc53315FfA3fcD1bc8b27d6c3155b0B56A,0xF23d8342881eDECcED51EA694AC21C2B68440929,1699030269,960470449280874,1699009671,91521104870607792451
0x972539E9d340a915775C004715f286a166F067Fd,0x85ec6ae01624ae0d2a04d0ffaad3a25884c7d0f30002...,80OVN/20wUSD+,0x99e2fafd901645fb611f6d56476af8ad25263ea5,0x972539E9d340a915775C004715f286a166F067Fd,0x0000000000000000000000000000000000000000,0,0,0,0
0x96056DF5a61B758005a219a052D72085C45df92c,0x8d333f82e0693f53fa48c40d5d4547142e907e1d0002...,80PAL-20OHM,0xd758454bdf4df7ad85f7538dc9742648ef8e6d0a,0x96056DF5a61B758005a219a052D72085C45df92c,0xF23d8342881eDECcED51EA694AC21C2B68440929,1698973692,499742937695466,1698938213,9577033768813232
0xcF853F14EF6111435Cb39c0C43C66366cc6300F1,0x2e8ea681fd59c9dc5f32b29de31f782724ef4dcb0001...,50GOLD-25USDC-25WSTETH,0xa8d4b31225bd6faf1363db5a0ab6c016894985d1,0xcF853F14EF6111435Cb39c0C43C66366cc6300F1,0x0000000000000000000000000000000000000000,0,0,0,0
0xd956246EA5b06DEa930F0A7feC1FFf000436e3f2,0x8bc65eed474d1a00555825c91feab6a8255c21070000...,DOLA/USDC BPT,0x25869f277f474fa9459f40f5d4cb0d6a8ab41967,0xd956246EA5b06DEa930F0A7feC1FFf000436e3f2,0xF23d8342881eDECcED51EA694AC21C2B68440929,1699030269,5787037037037037,1699002166,29049138358726894
